##Feature Engineering Notebook

**Goal: create new features for demographics, coverage periods, reimbursement/beneficiary and other potential derived features

**Input ../data/interim/beneficiary_cleaned.csv

**Output ../data/processed/beneficiary_features.csv

In [11]:
import os 
import pandas as pd 
import numpy

INPUT_PATH = '../data/interim/beneficiary_cleaned.csv'
OUTPUT_PATH = '../data/processed/beneficiary_features'
AS_OF_DATE = '2020-12-31'

In [12]:
#Read In data and observe

df = pd.read_csv(INPUT_PATH)
print(df.shape)

df.head()



(349064, 36)


,Beneficiary Code,Birth_date,Date_of_Death,Gender,Race,End_Stage_Renal_Disease_Indicator,State_code,County_code,Number_of_months_covered_a,Numver_of_months_covered_b,...,OP_medicare_reimbursement,OP_bene_resp_amount,OP_primary,Carrier_medicare_reimb,Carrier_bene_amount,Carrier_annual_payer_reimb,AGE,total_reimbursement,total_beneficiary_amount,total_primary_payment
0,00000B48BCF4AD29,1970-01-01 00:00:00.019230901,NaN,2,5,0,10,260,12,12,...,1520.0,80.0,0.0,6260.0,1520.0,0.0,55,88780.0,4672.0,0.0
1,0000525AB30E4DEF,1970-01-01 00:00:00.019201001,NaN,2,1,0,31,300,12,12,...,1760.0,670.0,0.0,3830.0,1010.0,50.0,55,18850.0,3728.0,50.0
2,00009C897C3D8372,1970-01-01 00:00:00.019320101,NaN,1,1,Y,7,70,12,12,...,100.0,160.0,0.0,1540.0,280.0,60.0,55,39140.0,4536.0,60.0
3,0001168CE43BE51B,1970-01-01 00:00:00.019340901,NaN,2,1,0,6,200,12,12,...,0.0,0.0,0.0,0.0,0.0,0.0,55,0.0,0.0,0.0
4,0002E494BC87CE10,1970-01-01 00:00:00.019140701,NaN,1,2,0,5,200,2,2,...,0.0,0.0,0.0,0.0,0.0,0.0,55,0.0,0.0,0.0


In [13]:
#Creating helper functions 

def safe_divide(numer, denom, eps=1.0):
    return numer / (denom + eps)

def to_datetime(s):
    return pd.to_datetime(s, errors='coerce')

#defining chronic conditions columns

chronic_columns = ['Alzhiemers_or_senile',
    'Heart_Failure',
    'Chronic_Kidney',
    'Cancer',
    'COPD',
    'Depression',
    'Diabetes',
    'ischemic_Heart_Disease',
    'Osteoporosis',
    'Rheumatoid_Arthritis',
    'Stroke']

In [14]:
#Creating and Mapping Demographic Features

fe = df.copy()

#Age

if 'Birth_date' in fe:
    birth = pd.to_datetime(fe['Birth_date'])
    as_of = pd.Timestamp(AS_OF_DATE)
    fe['AGE'] = ((as_of - birth).dt.days // 365).astype("Int64")

#Is Dead

if "Date_of_Death" in fe:
    death = to_datetime(fe['Date_of_Death'])
    fe['Date_of_Death'] = death.notna().astype(int)

#Mapping Gender and Race for Human comprehension

sex_map = {1:"Male", 2: "Female"}

race_map = {1:"White", 2:"Black", 3:"Other",4:"Asian", 5: "Hispanic"}

if "Gender" in fe:
    fe["Gender"] = fe["Gender"].map(sex_map).astype("string")

if "Race" in fe:
    fe["Race"] = fe["Race"].map(race_map).astype("string")

fe[["AGE","Date_of_Death","Gender","Race"]].head(3)

,AGE,Date_of_Death,Gender,Race
0,51,0,Female,Hispanic
1,51,0,Female,White
2,51,0,Male,White


In [16]:
#Creating Coverage & Utilization features

pa = pd.to_numeric(fe.get("Number_of_months_covered_a", 0), errors="coerce").fillna(0)
pb = pd.to_numeric(fe.get("Numver_of_months_covered_b", 0), errors="coerce").fillna(0)
hmo = pd.to_numeric(fe.get("Number_of_months_HMO_coverage", 0), errors="coerce").fillna(0)
pd_m = pd.to_numeric(fe.get("Number_of_months_covered_d", 0), errors="coerce").fillna(0)

fe['total_coverage_months'] = pa + pb + hmo + pd_m

#Chronic Burden

if chronic_columns:
    fe['chronic_count'] = (
        fe[chronic_columns].apply(pd.to_numeric, errors="coerce").fillna(0).astype(int).sum(axis=1)
    )

#Reimbursement totals
ip = pd.to_numeric(fe.get('IP_reimbursement',0), errors='coerce').fillna(0)
op = pd.to_numeric(fe.get('OP_medicare_reimbursement',0), errors='coerce').fillna(0)
car = pd.to_numeric(fe.get('Carrier_medicare_reimb',0), errors='coerce').fillna(0)

total_reimb = ip + op + car

#Ratio features

fe["avg_reimb"] = safe_divide(total_reimb, fe['total_coverage_months']).astype(float)
fe["op_ratio"] = safe_divide(op, ip).astype(float)
fe["car_ratio"] = safe_divide(car, (op + ip)).astype(float)

fe[["total_coverage_months","chronic_count","avg_reimb","op_ratio","car_ratio"]].describe().T

,count,mean,std,min,25%,50%,75%,max
total_coverage_months,349064.0,31.031874,11.900086,0.000000,24.0,36.000000,36.000000,48.0
chronic_count,349064.0,19.775694,2.451995,11.000000,18.0,21.000000,22.000000,22.0
avg_reimb,349064.0,138.894367,675.673983,-60.000000,0.0,30.000000,99.200000,115800.0
op_ratio,349064.0,417.883978,1375.417411,-100.000000,0.0,0.004999,280.000000,56930.0
car_ratio,349064.0,176.413754,549.163544,-301.052632,0.0,0.882598,14.705882,12800.0
